In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pandas.io.sql as sqlio
import requests as re
from IPython.display import JSON
import json

In [3]:
flights = pd.read_csv('../../../data/df_carrier_train.csv')

In [4]:
airport_link = pd.read_csv('../../../data/airport_link.csv')

In [5]:
airport_link.drop(['Unnamed: 0'],axis=1, inplace=True)

In [6]:
airport_link.columns

Index(['origin_airport_id', 'origin_city_name', 'dest_airport_id',
       'dest_city_name'],
      dtype='object')

In [7]:
origin_airport = airport_link[['origin_airport_id','origin_city_name']]

In [8]:
dest_airport = airport_link[['dest_airport_id', 'dest_city_name']]

In [9]:
origins = origin_airport.drop_duplicates()
origins

,origin_airport_id,origin_city_name
0,13930,"Chicago, IL"
1,14747,"Seattle, WA"
2,12892,"Los Angeles, CA"
5,11057,"Charlotte, NC"
6,11292,"Denver, CO"
14,14107,"Phoenix, AZ"
15,14771,"San Francisco, CA"
16,10397,"Atlanta, GA"
26,11298,"Dallas/Fort Worth, TX"
27,12266,"Houston, TX"


In [10]:
dest = dest_airport.drop_duplicates()
dest

,dest_airport_id,dest_city_name
0,15356,"Trenton, NJ"
1,10423,"Austin, TX"
3,12892,"Los Angeles, CA"
4,13487,"Minneapolis, MN"
5,13930,"Chicago, IL"
...,...,...
174068,14905,"Santa Maria, CA"
174622,13087,"Las Vegas, NV"
176686,10918,"Cedar City, UT"
177802,12898,"Latrobe, PA"


In [11]:
flights = flights.drop('Unnamed: 0',axis=1)

In [12]:
flights

,origin_airport_id,dest_airport_id,mkt_carrier,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,carrier_perc_delay,departures_performed,seats,passengers,percent_filled,arr_delay
0,12892,10140,WN,10,11,2018,1235,120.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,6.0
1,12892,10140,WN,10,11,2018,1840,105.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,1.0
2,12892,10397,WN,10,11,2018,805,250.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-12.0
3,12892,10397,WN,10,11,2018,1045,260.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-20.0
4,12892,10423,WN,10,11,2018,1030,185.0,1242,3.454152,4.566613,19.68,77.0,11427.0,9850.0,86.199352,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,VX,31,1,2018,1508,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-9.0
3918191,14771,14679,VX,31,1,2018,1720,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-11.0
3918192,14771,14679,VX,31,1,2018,2055,83.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-14.0
3918193,12892,14771,VX,31,1,2018,2000,94.0,337,4.221989,2.348741,19.31,314.0,46354.0,38317.0,82.661690,-18.0


In [13]:
df = pd.merge(flights, 
                 origins, 
                 on ='origin_airport_id', 
                 how ='left')

In [14]:
df

,origin_airport_id,dest_airport_id,mkt_carrier,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,carrier_perc_delay,departures_performed,seats,passengers,percent_filled,arr_delay,origin_city_name
0,12892,10140,WN,10,11,2018,1235,120.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,6.0,"Los Angeles, CA"
1,12892,10140,WN,10,11,2018,1840,105.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,1.0,"Los Angeles, CA"
2,12892,10397,WN,10,11,2018,805,250.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-12.0,"Los Angeles, CA"
3,12892,10397,WN,10,11,2018,1045,260.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-20.0,"Los Angeles, CA"
4,12892,10423,WN,10,11,2018,1030,185.0,1242,3.454152,4.566613,19.68,77.0,11427.0,9850.0,86.199352,-12.0,"Los Angeles, CA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,VX,31,1,2018,1508,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-9.0,"San Francisco, CA"
3918191,14771,14679,VX,31,1,2018,1720,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-11.0,"San Francisco, CA"
3918192,14771,14679,VX,31,1,2018,2055,83.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-14.0,"San Francisco, CA"
3918193,12892,14771,VX,31,1,2018,2000,94.0,337,4.221989,2.348741,19.31,314.0,46354.0,38317.0,82.661690,-18.0,"Los Angeles, CA"


In [15]:
df = pd.merge(df, 
                 dest, 
                 on ='dest_airport_id', 
                 how ='left')

In [16]:
df

,origin_airport_id,dest_airport_id,mkt_carrier,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,carrier_perc_delay,departures_performed,seats,passengers,percent_filled,arr_delay,origin_city_name,dest_city_name
0,12892,10140,WN,10,11,2018,1235,120.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,6.0,"Los Angeles, CA","Albuquerque, NM"
1,12892,10140,WN,10,11,2018,1840,105.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,1.0,"Los Angeles, CA","Albuquerque, NM"
2,12892,10397,WN,10,11,2018,805,250.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-12.0,"Los Angeles, CA","Atlanta, GA"
3,12892,10397,WN,10,11,2018,1045,260.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-20.0,"Los Angeles, CA","Atlanta, GA"
4,12892,10423,WN,10,11,2018,1030,185.0,1242,3.454152,4.566613,19.68,77.0,11427.0,9850.0,86.199352,-12.0,"Los Angeles, CA","Austin, TX"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,VX,31,1,2018,1508,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-9.0,"San Francisco, CA","San Diego, CA"
3918191,14771,14679,VX,31,1,2018,1720,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-11.0,"San Francisco, CA","San Diego, CA"
3918192,14771,14679,VX,31,1,2018,2055,83.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-14.0,"San Francisco, CA","San Diego, CA"
3918193,12892,14771,VX,31,1,2018,2000,94.0,337,4.221989,2.348741,19.31,314.0,46354.0,38317.0,82.661690,-18.0,"Los Angeles, CA","San Francisco, CA"


In [17]:
flights = df.copy()

In [18]:
flights

,origin_airport_id,dest_airport_id,mkt_carrier,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,carrier_perc_delay,departures_performed,seats,passengers,percent_filled,arr_delay,origin_city_name,dest_city_name
0,12892,10140,WN,10,11,2018,1235,120.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,6.0,"Los Angeles, CA","Albuquerque, NM"
1,12892,10140,WN,10,11,2018,1840,105.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,1.0,"Los Angeles, CA","Albuquerque, NM"
2,12892,10397,WN,10,11,2018,805,250.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-12.0,"Los Angeles, CA","Atlanta, GA"
3,12892,10397,WN,10,11,2018,1045,260.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-20.0,"Los Angeles, CA","Atlanta, GA"
4,12892,10423,WN,10,11,2018,1030,185.0,1242,3.454152,4.566613,19.68,77.0,11427.0,9850.0,86.199352,-12.0,"Los Angeles, CA","Austin, TX"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,VX,31,1,2018,1508,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-9.0,"San Francisco, CA","San Diego, CA"
3918191,14771,14679,VX,31,1,2018,1720,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-11.0,"San Francisco, CA","San Diego, CA"
3918192,14771,14679,VX,31,1,2018,2055,83.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-14.0,"San Francisco, CA","San Diego, CA"
3918193,12892,14771,VX,31,1,2018,2000,94.0,337,4.221989,2.348741,19.31,314.0,46354.0,38317.0,82.661690,-18.0,"Los Angeles, CA","San Francisco, CA"


In [19]:
flights['origin_city_name'].unique()

array(['Los Angeles, CA', 'Phoenix, AZ', 'Denver, CO', 'Seattle, WA',
       'San Francisco, CA', 'Atlanta, GA', 'Charlotte, NC',
       'Dallas/Fort Worth, TX', 'Houston, TX', 'Chicago, IL'],
      dtype=object)

In [20]:
citieNames = []
for index, city in enumerate(flights.origin_city_name):
    if '/' in city:
        splits = city.split(sep='/')
        city = splits[0] + ',' + splits[-1].split(sep=',')[-1]
    city2 = city.replace(" ","")
    city3 = city2.replace(",", "")    
    citieNames.append(city3)

In [21]:
flights['origin_city'] = citieNames

In [22]:
flights

,origin_airport_id,dest_airport_id,mkt_carrier,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,carrier_perc_delay,departures_performed,seats,passengers,percent_filled,arr_delay,origin_city_name,dest_city_name,origin_city
0,12892,10140,WN,10,11,2018,1235,120.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,6.0,"Los Angeles, CA","Albuquerque, NM",LosAngelesCA
1,12892,10140,WN,10,11,2018,1840,105.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,1.0,"Los Angeles, CA","Albuquerque, NM",LosAngelesCA
2,12892,10397,WN,10,11,2018,805,250.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-12.0,"Los Angeles, CA","Atlanta, GA",LosAngelesCA
3,12892,10397,WN,10,11,2018,1045,260.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-20.0,"Los Angeles, CA","Atlanta, GA",LosAngelesCA
4,12892,10423,WN,10,11,2018,1030,185.0,1242,3.454152,4.566613,19.68,77.0,11427.0,9850.0,86.199352,-12.0,"Los Angeles, CA","Austin, TX",LosAngelesCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,14771,14679,VX,31,1,2018,1508,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-9.0,"San Francisco, CA","San Diego, CA",SanFranciscoCA
3918191,14771,14679,VX,31,1,2018,1720,87.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-11.0,"San Francisco, CA","San Diego, CA",SanFranciscoCA
3918192,14771,14679,VX,31,1,2018,2055,83.0,447,4.221989,2.348741,19.31,158.0,22954.0,15712.0,68.449943,-14.0,"San Francisco, CA","San Diego, CA",SanFranciscoCA
3918193,12892,14771,VX,31,1,2018,2000,94.0,337,4.221989,2.348741,19.31,314.0,46354.0,38317.0,82.661690,-18.0,"Los Angeles, CA","San Francisco, CA",LosAngelesCA


In [23]:
flights.head()

,origin_airport_id,dest_airport_id,mkt_carrier,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,carrier_perc_delay,departures_performed,seats,passengers,percent_filled,arr_delay,origin_city_name,dest_city_name,origin_city
0,12892,10140,WN,10,11,2018,1235,120.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,6.0,"Los Angeles, CA","Albuquerque, NM",LosAngelesCA
1,12892,10140,WN,10,11,2018,1840,105.0,677,3.454152,4.566613,19.68,71.0,10153.0,8002.0,78.814144,1.0,"Los Angeles, CA","Albuquerque, NM",LosAngelesCA
2,12892,10397,WN,10,11,2018,805,250.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-12.0,"Los Angeles, CA","Atlanta, GA",LosAngelesCA
3,12892,10397,WN,10,11,2018,1045,260.0,1947,3.454152,4.566613,19.68,87.0,14169.0,12425.0,87.691439,-20.0,"Los Angeles, CA","Atlanta, GA",LosAngelesCA
4,12892,10423,WN,10,11,2018,1030,185.0,1242,3.454152,4.566613,19.68,77.0,11427.0,9850.0,86.199352,-12.0,"Los Angeles, CA","Austin, TX",LosAngelesCA


In [24]:
flights.drop(['origin_city_name'],axis=1,inplace=True)

In [25]:
Fflights = flights.copy()

In [26]:
Fflights = Fflights.sort_values(['origin_city', 'year', 'month', 'day', 'crs_dep_time'])


In [27]:
Fflights

,origin_airport_id,dest_airport_id,mkt_carrier,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,carrier_perc_delay,departures_performed,seats,passengers,percent_filled,arr_delay,dest_city_name,origin_city
338803,10397,11292,WN,1,1,2018,530,205.0,1199,4.221989,4.566613,19.68,117.0,19291.0,16440.0,85.221088,-7.0,"Denver, CO",AtlantaGA
338837,10397,12953,WN,1,1,2018,600,125.0,762,4.221989,4.566613,19.68,141.0,20931.0,17463.0,83.431274,1.0,"New York, NY",AtlantaGA
338851,10397,13232,WN,1,1,2018,600,115.0,591,4.221989,4.566613,19.68,146.0,21998.0,19954.0,90.708246,49.0,"Chicago, IL",AtlantaGA
1265910,10397,13303,AA,1,1,2018,600,115.0,594,4.221989,7.880984,21.18,112.0,14336.0,11446.0,79.840960,-12.0,"Miami, FL",AtlantaGA
1266589,10397,14100,AA,1,1,2018,600,124.0,666,4.221989,7.880984,21.18,130.0,13595.0,10605.0,78.006620,-27.0,"Philadelphia, PA",AtlantaGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682925,14747,11298,AS,31,12,2019,2320,220.0,1660,5.605429,2.328271,18.29,90.0,15944.0,14748.0,92.498746,-19.0,"Dallas/Fort Worth, TX",SeattleWA
2051290,14747,12264,UA,31,12,2019,2330,280.0,2306,5.605429,7.982953,20.75,74.0,12577.0,11825.0,94.020832,-18.0,"Washington, DC",SeattleWA
3682770,14747,10299,AS,31,12,2019,2330,224.0,1448,5.605429,2.328271,18.29,444.0,70688.0,62702.0,88.702467,-3.0,"Anchorage, AK",SeattleWA
2050716,14747,12266,UA,31,12,2019,2359,248.0,1874,5.605429,7.982953,20.75,120.0,18875.0,17947.0,95.083444,-18.0,"Houston, TX",SeattleWA


In [28]:
Fflights

,origin_airport_id,dest_airport_id,mkt_carrier,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,carrier_delay,carrier_perc_delay,departures_performed,seats,passengers,percent_filled,arr_delay,dest_city_name,origin_city
338803,10397,11292,WN,1,1,2018,530,205.0,1199,4.221989,4.566613,19.68,117.0,19291.0,16440.0,85.221088,-7.0,"Denver, CO",AtlantaGA
338837,10397,12953,WN,1,1,2018,600,125.0,762,4.221989,4.566613,19.68,141.0,20931.0,17463.0,83.431274,1.0,"New York, NY",AtlantaGA
338851,10397,13232,WN,1,1,2018,600,115.0,591,4.221989,4.566613,19.68,146.0,21998.0,19954.0,90.708246,49.0,"Chicago, IL",AtlantaGA
1265910,10397,13303,AA,1,1,2018,600,115.0,594,4.221989,7.880984,21.18,112.0,14336.0,11446.0,79.840960,-12.0,"Miami, FL",AtlantaGA
1266589,10397,14100,AA,1,1,2018,600,124.0,666,4.221989,7.880984,21.18,130.0,13595.0,10605.0,78.006620,-27.0,"Philadelphia, PA",AtlantaGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682925,14747,11298,AS,31,12,2019,2320,220.0,1660,5.605429,2.328271,18.29,90.0,15944.0,14748.0,92.498746,-19.0,"Dallas/Fort Worth, TX",SeattleWA
2051290,14747,12264,UA,31,12,2019,2330,280.0,2306,5.605429,7.982953,20.75,74.0,12577.0,11825.0,94.020832,-18.0,"Washington, DC",SeattleWA
3682770,14747,10299,AS,31,12,2019,2330,224.0,1448,5.605429,2.328271,18.29,444.0,70688.0,62702.0,88.702467,-3.0,"Anchorage, AK",SeattleWA
2050716,14747,12266,UA,31,12,2019,2359,248.0,1874,5.605429,7.982953,20.75,120.0,18875.0,17947.0,95.083444,-18.0,"Houston, TX",SeattleWA


In [29]:
Windspeeds = np.empty(3918195, dtype=object)
weatherDesc = np.empty(3918195, dtype=object)
precipitations = np.empty(3918195, dtype=object)
visibilities = np.empty(3918195, dtype=object)

temp_city = 'AtlantaGA'
r = open("../../../data/weather/201801AtlantaGA.json", "r")
jsonfile = json.load(r)['data']['weather']

temp_month = 1
for i in range(0,len(Fflights)-1):
    row = Fflights.iloc[i,:]
    
    day = row.day
    month = row.month
    city = row.origin_city
    year = row.year
    time = row.crs_dep_time
    
    if month < 10:
        strmonth = '0' + str(month)
    else:
        strmonth = str(month)
        
    if city != temp_city or month != temp_month:
        r.close()
        r = open("../../../data/weather/"+str(year)+strmonth+city+".json", "r")
        try:
            jsonfile = json.load(r)['data']['weather']
        except:
            Windspeeds[i]= np.nan
            visibilities[i]= np.nan
            precipitations[i]= np.nan
            weatherDesc[i]=np.nan
            continue
        temp_city = city
        temp_month = month
    
    if time >= 2100:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][7]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][7]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][7]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][7]['weatherDesc'][0]['value'])
    elif time >= 1800:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][6]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][6]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][6]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][6]['weatherDesc'][0]['value'])
    elif time >= 1500:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][5]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][5]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][5]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][5]['weatherDesc'][0]['value'])
    elif time >= 1200:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][4]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][4]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][4]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][4]['weatherDesc'][0]['value'])
    elif time >= 900:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][3]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][3]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][3]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][3]['weatherDesc'][0]['value'])
    elif time >= 600:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][2]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][2]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][2]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][2]['weatherDesc'][0]['value'])
    elif time >= 300:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][1]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][1]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][1]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][1]['weatherDesc'][0]['value'])
    else:
        Windspeeds[i]=(jsonfile[day-1]['hourly'][0]['windspeedMiles'])
        visibilities[i]=(jsonfile[day-1]['hourly'][0]['visibility'])
        precipitations[i]=(jsonfile[day-1]['hourly'][0]['precipMM'])
        weatherDesc[i]=(jsonfile[day-1]['hourly'][0]['weatherDesc'][0]['value'])

KeyboardInterrupt: 

In [30]:
Windspeeds

array(['8', '7', '7', ..., None, None, None], dtype=object)

In [31]:
Windspeeds = pd.DataFrame(Windspeeds)

In [32]:
weatherDesc = pd.DataFrame(weatherDesc)

In [33]:
visibilities = pd.DataFrame(visibilities)

In [34]:
precipitations = pd.DataFrame(precipitations)

In [35]:
precipitations.isna().sum()

0    3792873
dtype: int64

In [36]:
Fflights['windspeeds'] = Windspeeds

In [37]:
Fflights['weather_desc'] = weatherDesc

In [38]:
Fflights['visibilities'] = visibilities

In [39]:
Fflights['precipitations'] = precipitations

In [40]:
Fflights.columns

Index(['origin_airport_id', 'dest_airport_id', 'mkt_carrier', 'day', 'month',
       'year', 'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'arr_delay', 'dest_city_name',
       'origin_city', 'windspeeds', 'weather_desc', 'visibilities',
       'precipitations'],
      dtype='object')

In [115]:
Fflights.to_csv('../../../data/final_flights_unclean.csv')

array([7, 4, 1])

In [51]:
train_flight = pd.read_csv('../../../data/final_flights_unclean.csv')

In [84]:
train_flight = train_flight.sort_values(['origin_airport_id', 'dest_airport_id', 'day','month','year']).reset_index()

In [85]:
Fflights = Fflights.sort_values(['origin_airport_id', 'dest_airport_id', 'day','month','year']).reset_index()

In [ ]:
train_flight = train_flight.set_index(['Unnamed: 0'])

In [86]:
train_flight['mkt_carrier'] = Fflights['mkt_carrier']

In [87]:
Fflights

,index,origin_airport_id,dest_airport_id,mkt_carrier,day,month,year,crs_dep_time,crs_elapsed_time,distance,...,seats,passengers,percent_filled,arr_delay,dest_city_name,origin_city,windspeeds,weather_desc,visibilities,precipitations
0,3136323,10397,10135,DL,1,1,2018,1505,125.0,692,...,3300.0,2018.0,61.151515,110.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,None,None,None,None
1,3102344,10397,10135,DL,1,1,2019,1746,121.0,692,...,2970.0,2285.0,76.936027,-16.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,None,None,None,None
2,2850117,10397,10135,DL,1,2,2018,1015,124.0,692,...,3850.0,2667.0,69.272727,-20.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,None,None,None,None
3,2850113,10397,10135,DL,1,2,2018,1515,124.0,692,...,3850.0,2667.0,69.272727,-12.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,None,None,None,None
4,2850748,10397,10135,DL,1,2,2018,2130,127.0,692,...,3850.0,2667.0,69.272727,-8.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,2090181,14771,15376,UA,29,10,2018,1905,131.0,751,...,534.0,435.0,81.460674,-14.0,"Tucson, AZ",SanFranciscoCA,None,None,None,None
3918191,2091646,14771,15376,UA,30,10,2018,830,137.0,751,...,534.0,435.0,81.460674,-15.0,"Tucson, AZ",SanFranciscoCA,None,None,None,None
3918192,2091675,14771,15376,UA,30,10,2018,1905,131.0,751,...,534.0,435.0,81.460674,-9.0,"Tucson, AZ",SanFranciscoCA,None,None,None,None
3918193,2092757,14771,15376,UA,31,10,2018,830,137.0,751,...,534.0,435.0,81.460674,3.0,"Tucson, AZ",SanFranciscoCA,None,None,None,None


In [94]:
train_flight.sort_values(['origin_airport_id', 'dest_airport_id', 'day','month','year'])

,Unnamed: 0,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,...,carrier_VX,carrier_WN,arr_delay,dest_city_name,origin_city,windspeeds,weather_desc,visibilities,precipitations,mkt_carrier
0,3136323,10397,10135,1,1,2018,1505,125.0,692,4.221989,...,0,0,110.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,1.0,Sunny,10.0,0.0,DL
1,3102344,10397,10135,1,1,2019,1746,121.0,692,4.221989,...,0,0,-16.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,5.0,Sunny,10.0,0.0,DL
2,2850117,10397,10135,1,2,2018,1015,124.0,692,6.956543,...,0,0,-20.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,6.0,Clear,10.0,0.0,DL
3,2850113,10397,10135,1,2,2018,1515,124.0,692,6.956543,...,0,0,-12.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,6.0,Clear,10.0,0.0,DL
4,2850748,10397,10135,1,2,2018,2130,127.0,692,6.956543,...,0,0,-8.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,4.0,Clear,10.0,0.0,DL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,2090181,14771,15376,29,10,2018,1905,131.0,751,3.764646,...,0,0,-14.0,"Tucson, AZ",SanFranciscoCA,9.0,Sunny,10.0,0.0,UA
3918191,2091646,14771,15376,30,10,2018,830,137.0,751,3.764646,...,0,0,-15.0,"Tucson, AZ",SanFranciscoCA,2.0,Sunny,10.0,0.0,UA
3918192,2091675,14771,15376,30,10,2018,1905,131.0,751,3.764646,...,0,0,-9.0,"Tucson, AZ",SanFranciscoCA,2.0,Sunny,10.0,0.0,UA
3918193,2092757,14771,15376,31,10,2018,830,137.0,751,3.764646,...,0,0,3.0,"Tucson, AZ",SanFranciscoCA,8.0,Patchy heavy snow,5.0,1.4,UA


In [106]:
train_flights = train_flight.copy()
# train_flights = Fflights.copy()

In [107]:
c1 =['Clear', 'Sunny']
c2=['Light drizzle', 'Cloudy', 'Partly cloudy','Patchy rain possible','Light rain shower', 'Light rain','Moderate rain at times', 'Patchy light rain']
c3=['Moderate rain',  'Moderate or heavy rain shower', 'Patchy light drizzle', 'Mist','Patchy light rain with thunder','Fog', 'Overcast']
c4=['Light sleet', 'Light snow','Light freezing rain','Patchy light snow','Light snow showers','Patchy snow possible','Patchy sleet possible','Light sleet showers']
c5=['Ice pellets', 'Patchy moderate snow', 'Thundery outbreaks possible', 'Moderate snow']
c6=['Patchy heavy snow', 'Moderate or heavy freezing rain','Moderate or heavy sleet','Moderate or heavy snow showers','Heavy rain at times','Blowing snow', 'Heavy snow', 'Heavy rain']
c7=['Moderate or heavy rain with thunder','Moderate or heavy snow with thunder', 'Heavy freezing drizzle']
c8=['Blizzard','Torrential rain shower', 'Freezing fog']


c=[c1,c2,c3,c4,c5,c6,c7,c8]  
v=[0,1,2,3,4,5,6,7]

## Binning of weather data

In [108]:
for index,value in enumerate(c):
    train_flights['weather_desc'].replace(value,index, inplace=True)

In [109]:
train_flights

,Unnamed: 0,origin_airport_id,dest_airport_id,day,month,year,crs_dep_time,crs_elapsed_time,distance,month_delay,...,carrier_VX,carrier_WN,arr_delay,dest_city_name,origin_city,windspeeds,weather_desc,visibilities,precipitations,mkt_carrier
0,3136323,10397,10135,1,1,2018,1505,125.0,692,4.221989,...,0,0,110.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,1.0,0.0,10.0,0.0,DL
1,3102344,10397,10135,1,1,2019,1746,121.0,692,4.221989,...,0,0,-16.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,5.0,0.0,10.0,0.0,DL
2,2850117,10397,10135,1,2,2018,1015,124.0,692,6.956543,...,0,0,-20.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,6.0,0.0,10.0,0.0,DL
3,2850113,10397,10135,1,2,2018,1515,124.0,692,6.956543,...,0,0,-12.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,6.0,0.0,10.0,0.0,DL
4,2850748,10397,10135,1,2,2018,2130,127.0,692,6.956543,...,0,0,-8.0,"Allentown/Bethlehem/Easton, PA",AtlantaGA,4.0,0.0,10.0,0.0,DL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918190,2090181,14771,15376,29,10,2018,1905,131.0,751,3.764646,...,0,0,-14.0,"Tucson, AZ",SanFranciscoCA,9.0,0.0,10.0,0.0,UA
3918191,2091646,14771,15376,30,10,2018,830,137.0,751,3.764646,...,0,0,-15.0,"Tucson, AZ",SanFranciscoCA,2.0,0.0,10.0,0.0,UA
3918192,2091675,14771,15376,30,10,2018,1905,131.0,751,3.764646,...,0,0,-9.0,"Tucson, AZ",SanFranciscoCA,2.0,0.0,10.0,0.0,UA
3918193,2092757,14771,15376,31,10,2018,830,137.0,751,3.764646,...,0,0,3.0,"Tucson, AZ",SanFranciscoCA,8.0,5.0,5.0,1.4,UA


In [110]:
train_flights.columns

Index(['Unnamed: 0', 'origin_airport_id', 'dest_airport_id', 'day', 'month',
       'year', 'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN', 'arr_delay', 'dest_city_name',
       'origin_city', 'windspeeds', 'weather_desc', 'visibilities',
       'precipitations', 'mkt_carrier'],
      dtype='object')

In [111]:
train_flights.drop(['Unnamed: 0','origin_city', 'dest_city_name','carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN'], inplace=True,axis=1)

In [112]:
tf_final = train_flights.dropna()

In [113]:
tf_final.shape

(3898337, 21)

In [114]:
tf_final.to_csv('../../../data/training_set/TRAIN_ME_carriers.csv')